# Test set

##### Test set에 적용

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns

### 1. Data load

In [6]:
# DATA_IN_PATH = 'C:/Users/이혁주/Desktop/FinTech/give_me_some_credit/datasets/'
# INPUT_TRAIN_DATA_FILE = 'train_data_X_2.npy'
# LABEL_TRAIN_DATA_FILE = 'train_data_y_2.npy'

# train_input = np.load(open(DATA_IN_PATH + INPUT_TRAIN_DATA_FILE, 'rb'))
# train_label = np.load(open(DATA_IN_PATH + LABEL_TRAIN_DATA_FILE, 'rb'))
# print(train_input.shape)
# print(train_label.shape)

(150000, 10)
(150000,)


In [9]:
train_df = pd.read_csv("C:/python/fintech_credit/cs-training.csv")
train_df['NumberOfDependents'].fillna(train_df['NumberOfDependents'].median(), inplace=True)
train_df['MonthlyIncome'].fillna(train_df['MonthlyIncome'].median(), inplace=True)
train_df.NumberRealEstateLoansOrLines[30587] = 32
for i in train_df[train_df['RevolvingUtilizationOfUnsecuredLines'] > 10000].index:
    train_df['RevolvingUtilizationOfUnsecuredLines'][i] = 9684
train_y = train_df['SeriousDlqin2yrs']
train_X = train_df.drop(['Unnamed: 0', 'SeriousDlqin2yrs'], axis=1)
print(train_X.shape)
print(train_y.shape)

C:\Anaconda_\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda_\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(150000, 10)
(150000,)


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)

In [11]:
train_X = np.array(train_X)
train_y = np.array(train_y)

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
GB = GradientBoostingClassifier(random_state=42)
GB.fit(train_X, train_y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

### 2. Test set processing & predict

In [15]:
test_df = pd.read_csv("C:/python/fintech_credit/cs-test.csv")
test_df = test_df.drop(['Unnamed: 0', 'SeriousDlqin2yrs'], axis=1)
test_df['NumberOfDependents'].fillna(train_df['NumberOfDependents'].median(), inplace=True)
test_df['MonthlyIncome'].fillna(train_df['MonthlyIncome'].median(), inplace=True)

In [16]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101503 entries, 0 to 101502
Data columns (total 10 columns):
RevolvingUtilizationOfUnsecuredLines    101503 non-null float64
age                                     101503 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    101503 non-null int64
DebtRatio                               101503 non-null float64
MonthlyIncome                           101503 non-null float64
NumberOfOpenCreditLinesAndLoans         101503 non-null int64
NumberOfTimes90DaysLate                 101503 non-null int64
NumberRealEstateLoansOrLines            101503 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    101503 non-null int64
NumberOfDependents                      101503 non-null float64
dtypes: float64(4), int64(6)
memory usage: 7.7 MB


In [17]:
test_df = scaler.transform(test_df)

In [18]:
test_df.shape

(101503, 10)

In [19]:
prediction = GB.predict_proba(test_df)
probability = prediction[:, 1]
probability.shape

(101503,)

In [20]:
test_df = pd.read_csv("C:/python/fintech_credit/cs-test.csv")
test_X = test_df['Unnamed: 0']
submit = pd.DataFrame({'ID': test_X, 'Probability': probability})
submit.to_csv("C:/python/fintech_credit/submit.csv", index=False)